**🔧 Setup Required**: Before running this notebook, please follow the [setup instructions](../README.md#setup-instructions) to configure your environment and API keys.

# Building a Routing Pipeline with Haystack

Welcome to this notebook where we'll explore intelligent query routing in Haystack. Routing is a powerful pattern that allows you to direct queries to different processing paths based on their characteristics, improving both efficiency and accuracy.

## What You'll Learn

- How to implement conditional routing based on query analysis
- Different routing strategies: keyword-based, metadata-based, and zero-shot classification
- How to create specialized processing branches for different query types
- Building a multi-path pipeline that adapts to different user needs
- Practical routing patterns for production RAG systems

## Why Routing Matters

Not all queries are created equal. Some queries:
- Require factual lookup vs. conceptual explanation
- Need current information vs. historical data
- Are best served by different retrieval strategies
- Benefit from specialized processing pipelines

Routing allows your system to intelligently choose the best approach for each query, optimizing both cost and quality.

## 1. Core Routing Components

Let's start by importing the essential routing components from Haystack:

- **`ConditionalRouter`**: Routes data based on custom conditions and rules
- **`TextLanguageRouter`**: Routes queries based on detected language
- **`FileTypeRouter`**: Routes files based on their MIME type
- **`MetadataRouter`**: Routes documents based on metadata fields

We'll also import the basic RAG components we've used in previous notebooks to build complete pipelines for each route.

In [ ]:
# Import routing components
from haystack.components.routers import ConditionalRouter, TextLanguageRouter, MetadataRouter


/Users/laurafunderburk/Documents/GitHub/Building-Natural-Language-Pipelines/ch4/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Understanding ConditionalRouter

The `ConditionalRouter` is the most flexible routing component. It evaluates conditions against input data and routes to different outputs based on the results.

### Key Features

- **Jinja2 Expressions**: Use template syntax for complex conditions
- **Multiple Routes**: Define as many output paths as needed
- **Flexible Logic**: Combine multiple conditions with AND/OR logic
- **Metadata Access**: Route based on any available metadata

### Common Use Cases

1. **Query Type Detection**: Route factual vs. conceptual questions
2. **Keyword Matching**: Direct queries containing specific terms
3. **Length-Based Routing**: Handle short vs. long queries differently
4. **Metadata Filtering**: Route based on query metadata

### Important: Default Routes

Always include a default/fallback route with `"condition": "{{ True }}"` as the last route. This ensures that queries not matching any specific condition are still handled gracefully, preventing `NoRouteSelectedException` errors.

Let's create a simple example that routes queries based on keywords.

In [3]:
# Example 1: Simple Keyword-Based Routing
# This router will direct queries to different paths based on content

# Define routing rules using Jinja2 template syntax
routes = [
    {
        "condition": "{{ 'technical' in query.lower() or 'api' in query.lower() or 'code' in query.lower() }}",
        "output": "{{ query }}",
        "output_name": "technical",
        "output_type": str,
    },
    {
        "condition": "{{ 'concept' in query.lower() or 'explain' in query.lower() or 'what is' in query.lower() }}",
        "output": "{{ query }}",
        "output_name": "conceptual",
        "output_type": str,
    },
    {
        "condition": "{{ 'how to' in query.lower() or 'tutorial' in query.lower() }}",
        "output": "{{ query }}",
        "output_name": "tutorial",
        "output_type": str,
    },
    {
        # Default route: catch all queries that don't match other conditions
        "condition": "{{ True }}",
        "output": "{{ query }}",
        "output_name": "general",
        "output_type": str,
    },
]

# Create the router
query_router = ConditionalRouter(routes=routes)

# Test the router
test_queries = [
    "How do I use the technical API?",
    "Explain what is Haystack?",
    "Show me a tutorial on building pipelines",
    "What are the best practices?"
]

print("Testing Query Router:\n")
for query in test_queries:
    result = query_router.run(query=query)
    route = list(result.keys())[0] if result else "no_match"
    print(f"Query: '{query}'")
    print(f"Routed to: {route}\n")

Testing Query Router:

Query: 'How do I use the technical API?'
Routed to: technical

Query: 'Explain what is Haystack?'
Routed to: conceptual

Query: 'Show me a tutorial on building pipelines'
Routed to: tutorial

Query: 'What are the best practices?'
Routed to: general



## 3. Building a Query Classification Router

For more sophisticated routing, we can use the query type to determine the best processing strategy. Let's create a router that distinguishes between:

1. **Factual Queries**: Direct questions seeking specific information
   - Best served by: BM25 (keyword matching)
   - Example: "What is the release date of Haystack 2.0?"

2. **Semantic Queries**: Questions about concepts or relationships
   - Best served by: Dense embeddings (semantic search)
   - Example: "How does Haystack compare to other frameworks?"

3. **Complex Queries**: Multi-faceted questions
   - Best served by: Hybrid search (both methods)
   - Example: "What are the technical capabilities and use cases?"

This approach optimizes both speed and accuracy by matching queries to the most appropriate retrieval method.

In [4]:
# Define routes based on query characteristics
classification_routes = [
    {
        # Factual queries: specific facts, names, dates, numbers
        "condition": "{{ 'when' in query.lower() or 'who' in query.lower() or 'what is the' in query.lower() }}",
        "output": "{{ query }}",
        "output_name": "factual",
        "output_type": str,
    },
    {
        # Semantic queries: understanding, comparison, explanation
        "condition": "{{ 'how does' in query.lower() or 'compare' in query.lower() or 'difference between' in query.lower() }}",
        "output": "{{ query }}",
        "output_name": "semantic",
        "output_type": str,
    },
    {
        # Complex queries: multiple aspects or comprehensive information
        "condition": "{{ 'and' in query.lower() or len(query.split()) > 10 }}",
        "output": "{{ query }}",
        "output_name": "complex",
        "output_type": str,
    },
    {
        # Default route: catch-all for queries that don't match specific patterns
        "condition": "{{ True }}",
        "output": "{{ query }}",
        "output_name": "complex",  # Default to complex for comprehensive handling
        "output_type": str,
    },
]

classification_router = ConditionalRouter(routes=classification_routes)

# Test with different query types
test_classification = [
    "When was Haystack 2.0 released?",
    "How does Haystack compare to LangChain?",
    "What are the main features and benefits of using Haystack for production applications?"
]

print("Testing Query Classification Router:\n")
for query in test_classification:
    result = classification_router.run(query=query)
    route = list(result.keys())[0] if result else "no_match"
    print(f"Query: '{query}'")
    print(f"Classified as: {route}\n")

Testing Query Classification Router:

Query: 'When was Haystack 2.0 released?'
Classified as: factual

Query: 'How does Haystack compare to LangChain?'
Classified as: semantic

Query: 'What are the main features and benefits of using Haystack for production applications?'
Classified as: complex



## 4. TextLanguageRouter - Multilingual Routing

The `TextLanguageRouter` detects the language of input text and routes it to language-specific processing paths. This is essential for multilingual applications where different languages may need:

- Different embedding models
- Language-specific retrievers
- Translation pipelines
- Region-specific knowledge bases

### How It Works

1. Uses language detection to identify the input language
2. Routes text to predefined language-specific outputs
3. Handles unknown languages with fallback behavior

Let's create a simple example that routes text based on detected language.

In [ ]:
# Example: TextLanguageRouter for multilingual support
from haystack import Pipeline

# Create a language router
language_router = TextLanguageRouter()

# Test with queries in different languages
multilingual_queries = [
    "What is Haystack?",  # English
    "¿Qué es Haystack?",  # Spanish
    "Qu'est-ce que Haystack?",  # French
    "Was ist Haystack?",  # German
    "Haystack是什么？",  # Chinese
]

print("Testing TextLanguageRouter:\n")
for query in multilingual_queries:
    result = language_router.run(text=query)
    # The router outputs to language-specific keys (e.g., 'en', 'es', 'fr')
    route = list(result.keys())[0] if result else "unknown"
    print(f"Query: '{query}'")
    print(f"Detected language: {route}")
    print(f"Routed text: {result.get(route, 'N/A')}\n")

### Visualizing the TextLanguageRouter Pipeline

Let's create a pipeline with the language router and visualize how queries flow through different language-specific paths.

In [ ]:
# Build a simple pipeline with language routing and visualization
from haystack.components.builders import PromptBuilder

# Create a simple pipeline that routes by language
language_pipeline = Pipeline()

# Add the language router
language_pipeline.add_component("language_router", TextLanguageRouter())

# Add language-specific prompt builders
en_template = """
You are processing an English query: {{ query }}
Language: English
"""

es_template = """
Estás procesando una consulta en español: {{ query }}
Idioma: Español
"""

fr_template = """
Vous traitez une requête en français: {{ query }}
Langue: Français
"""

en_prompt = PromptBuilder(template=en_template)
es_prompt = PromptBuilder(template=es_template)
fr_prompt = PromptBuilder(template=fr_template)

language_pipeline.add_component("en_prompt", en_prompt)
language_pipeline.add_component("es_prompt", es_prompt)
language_pipeline.add_component("fr_prompt", fr_prompt)

# Connect the router to language-specific components
language_pipeline.connect("language_router.en", "en_prompt.query")
language_pipeline.connect("language_router.es", "es_prompt.query")
language_pipeline.connect("language_router.fr", "fr_prompt.query")

# Visualize the pipeline
print("Language Routing Pipeline Structure:\n")
language_pipeline.draw("jupyter-notebooks/images/language_routing_pipeline.png")
print("Pipeline diagram saved to: jupyter-notebooks/images/language_routing_pipeline.png")

# Display the pipeline
try:
    from IPython.display import Image, display
    display(Image("jupyter-notebooks/images/language_routing_pipeline.png"))
except Exception as e:
    print(f"Could not display image: {e}")

## 5. MetadataRouter - Document Filtering by Metadata

The `MetadataRouter` is specifically designed to route documents based on their metadata fields. Unlike the `ConditionalRouter` which can work with any data, the `MetadataRouter` is optimized for document filtering scenarios.

### Key Use Cases

- Filter documents by category, source, or date
- Route documents to specialized processors
- Split document collections by type
- Implement access control based on metadata

### How It Works

The `MetadataRouter` examines document metadata and routes documents to different outputs based on rules you define. It's particularly useful in RAG pipelines where you want to process different document types differently.

Let's create a practical example.

## 5. Advanced Routing: Metadata-Based Routing

Beyond query content, we can also route based on document metadata. This is useful when:

- Documents have category tags
- Content is organized by source or date
- Different document types need specialized handling
- User preferences or permissions affect routing

Let's demonstrate metadata routing with a simple example. The `MetadataRouter` can filter documents based on metadata fields before retrieval or processing.

In [6]:
# Example: Metadata-based routing
# Create documents with different metadata
from haystack import Document

sample_docs = [
    Document(content="Technical documentation about APIs", meta={"type": "technical", "category": "api"}),
    Document(content="Conceptual guide to understanding LLMs", meta={"type": "conceptual", "category": "guide"}),
    Document(content="Step-by-step tutorial", meta={"type": "tutorial", "category": "howto"}),
]

# Create a metadata router
metadata_routes = [
    {
        "condition": "{{ meta['type'] == 'technical' }}",
        "output": "{{ documents }}",
        "output_name": "technical_docs",
        "output_type": list,
    },
    {
        "condition": "{{ meta['type'] == 'conceptual' }}",
        "output": "{{ documents }}",
        "output_name": "conceptual_docs",
        "output_type": list,
    },
    {
        "condition": "{{ meta['type'] == 'tutorial' }}",
        "output": "{{ documents }}",
        "output_name": "tutorial_docs",
        "output_type": list,
    },
    {
        # Default route: catch-all for unmatched document types
        "condition": "{{ True }}",
        "output": "{{ documents }}",
        "output_name": "other_docs",
        "output_type": list,
    },
]

metadata_router = ConditionalRouter(routes=metadata_routes)

# Test metadata routing
print("\nTesting Metadata-Based Routing:\n")
for doc in sample_docs:
    result = metadata_router.run(documents=[doc], meta=doc.meta)
    route = list(result.keys())[0] if result else "no_match"
    print(f"Document: '{doc.content}'")
    print(f"Metadata: {doc.meta}")
    print(f"Routed to: {route}\n")


Testing Metadata-Based Routing:

Document: 'Technical documentation about APIs'
Metadata: {'type': 'technical', 'category': 'api'}
Routed to: technical_docs

Document: 'Conceptual guide to understanding LLMs'
Metadata: {'type': 'conceptual', 'category': 'guide'}
Routed to: conceptual_docs

Document: 'Step-by-step tutorial'
Metadata: {'type': 'tutorial', 'category': 'howto'}
Routed to: tutorial_docs



## 7. Summary and Next Steps

Congratulations! You've learned how to build sophisticated routing systems in Haystack. Let's recap:

### What We Covered

- ✅ Basic routing with ConditionalRouter
- ✅ Query classification and route selection
- ✅ Building multi-path RAG pipelines
- ✅ Metadata-based routing strategies
- ✅ Best practices for production systems

### Key Takeaways

1. Routing enables query-specific optimization
2. Different retrieval methods suit different query types
3. Metadata can guide intelligent document selection
4. Modular design allows independent route optimization

### Next Steps

- Experiment with your own routing conditions
- Add more specialized routes for your use case
- Implement monitoring and metrics per route
- Explore zero-shot classification for routing
- Build adaptive routes that learn from user feedback

### Further Exploration

- Combine routing with the hybrid pipeline from the previous notebook
- Implement cost-based routing (cheap vs. expensive models)
- Add language detection for multilingual routing
- Create user-preference-based routing

Happy routing! 🚀